## Visualizing and Exploring NYC Restaurant Inspection Data with Altair


### Table of Contents
- [Cleaning of the data](#clean)
- [Latest inspection grade by borough](#boro)
- [Number of Inspections over time](#ts) 
- [Map of Restaurants with poor sanitation grade](#map_viz)
- [Top 10 Cuisines and Inspections](#top_10)



<a id='clean'></a>
### Import NYC Restaurant Inspection Dataset
We retrieved the latest NYC Restaurant Inspection Data from NYC Open Data, data as of 12/16/2019.

In [1]:
import pandas as pd
import numpy as np
df_rest = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
df_rest.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50064946,RAVIOLO,Manhattan,57,7TH AVE S,10014.0,2124330818,Italian,10/12/2017,Violations were cited in the following area(s).,...,11/25/2019,Pre-permit (Operational) / Initial Inspection,40.731524,-74.003983,102.0,3.0,6700.0,1009946.0,1.005870e+09,MN23
1,50000342,BLEND ASTORIA,Queens,3717,30TH AVENUE,11103.0,3478134437,"Latin (Cuban, Dominican, Puerto Rican, South &...",03/29/2018,Violations were cited in the following area(s).,...,11/25/2019,Cycle Inspection / Initial Inspection,40.764543,-73.916371,401.0,22.0,6502.0,4010727.0,4.006610e+09,QN70
2,50040738,HAAGEN-DAZS,Queens,9015,QUEENS BLVD,11373.0,7187607220,American,08/05/2016,Establishment re-opened by DOHMH,...,11/25/2019,Cycle Inspection / Reopening Inspection,40.733840,-73.871577,404.0,25.0,68300.0,4045999.0,4.018600e+09,QN29
3,40797688,TRIPLE CROWN DINER,Queens,24827,JAMAICA AVENUE,11426.0,7183474600,American,10/23/2018,Violations were cited in the following area(s).,...,11/25/2019,Cycle Inspection / Initial Inspection,40.726252,-73.717491,413.0,23.0,161700.0,4177065.0,4.086640e+09,QN43
4,50068466,UNTAMED SANDWICHES,Manhattan,643,LEXINGTON AVE,10022.0,6468821877,Sandwiches,12/21/2018,No violations were recorded at the time of thi...,...,11/25/2019,Administrative Miscellaneous / Initial Inspection,40.759229,-73.970350,106.0,4.0,10000.0,1036487.0,1.013090e+09,MN19


In [2]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397073 entries, 0 to 397072
Data columns (total 26 columns):
CAMIS                    397073 non-null int64
DBA                      396400 non-null object
BORO                     397073 non-null object
BUILDING                 396820 non-null object
STREET                   397062 non-null object
ZIPCODE                  391506 non-null float64
PHONE                    397056 non-null object
CUISINE DESCRIPTION      397073 non-null object
INSPECTION DATE          397073 non-null object
ACTION                   395368 non-null object
VIOLATION CODE           390979 non-null object
VIOLATION DESCRIPTION    387719 non-null object
CRITICAL FLAG            387719 non-null object
SCORE                    379860 non-null float64
GRADE                    200599 non-null object
GRADE DATE               199037 non-null object
RECORD DATE              397073 non-null object
INSPECTION TYPE          395368 non-null object
Latitude                

In [3]:
# format date fields as datetime
date_cols = ['RECORD DATE', 'INSPECTION DATE']

for col in date_cols:
    df_rest[col] = pd.to_datetime(df_rest[col])

In [4]:
# # drop restaurants that don't have restaurant grade ABC 
# df_rest = df_rest[df_rest['GRADE'].isin(['A','B','C'])]

In [5]:
# drop records where lat/lng are nulls
df_rest = df_rest[(df_rest['Latitude'].notnull() & df_rest['Longitude'].notnull())]

# drop records where lat/long are 0
df_rest = df_rest[(df_rest['Latitude'] != 0) | (df_rest['Longitude'] != 0)]

In [6]:
# drop restaurants where DBA (name of restaurant) is null
df_rest = df_rest[df_rest['DBA'].notnull()]

In [7]:
# check for number of unique restaurants
df_rest['CAMIS'].nunique()

26040

In [8]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390422 entries, 0 to 397072
Data columns (total 26 columns):
CAMIS                    390422 non-null int64
DBA                      390422 non-null object
BORO                     390422 non-null object
BUILDING                 390416 non-null object
STREET                   390422 non-null object
ZIPCODE                  390422 non-null float64
PHONE                    390405 non-null object
CUISINE DESCRIPTION      390422 non-null object
INSPECTION DATE          390422 non-null datetime64[ns]
ACTION                   389411 non-null object
VIOLATION CODE           385145 non-null object
VIOLATION DESCRIPTION    381937 non-null object
CRITICAL FLAG            381937 non-null object
SCORE                    374141 non-null float64
GRADE                    197381 non-null object
GRADE DATE               195822 non-null object
RECORD DATE              390422 non-null datetime64[ns]
INSPECTION TYPE          389411 non-null object
Latitude

<a id='boro'></a>
### Let's visualize the number of restaurants by the latest inspection grade by borough
We've filtered to the latast inspection for each restaurant and counted them by borough and grade. 
We also looked at the proportion of inspection grade within each borough.

In [9]:
import altair as alt
from vega_datasets import data

In [10]:
latest_inspection = df_rest.sort_values(['CAMIS', 'INSPECTION DATE'], 
                    ascending=[True, False]).groupby('CAMIS').head(1)

grade_count = latest_inspection.groupby(['BORO', 'GRADE'])['CAMIS'].nunique().reset_index()

chart1 = alt.Chart(grade_count).mark_bar(cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3).encode(
    x='GRADE',
    y='CAMIS',
    color='GRADE',
    column='BORO').properties(
    height=200,
    width=100
)

chart2 =alt.Chart(grade_count).mark_bar().encode(
    x=alt.X('sum(CAMIS)', stack="normalize"),
    y='BORO',
    color='GRADE').properties(
    height=200,
    width=600
)

alt.vconcat(chart1, chart2)

alt.VConcatChart(...)

<a id='ts'></a>
### Let's visualize the number of restaurants by grade over time by Inspection Date

In [11]:
#create timeseries data of monthly inspections 
df_dates = df_rest.copy()
df_dates.index = df_dates['INSPECTION DATE']
grades_ts = df_dates.groupby('GRADE').resample('MS')['CAMIS'].count().reset_index()

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['INSPECTION DATE'], empty='none')

# Line Chart
line = alt.Chart().mark_line(interpolate='basis').encode(
    alt.X('INSPECTION DATE:T', axis=alt.Axis(title='')),
    alt.Y('CAMIS:Q', axis=alt.Axis(title='')),
    color='GRADE:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart().mark_point().encode(
    x='INSPECTION DATE:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'CAMIS:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart().mark_rule(color='gray').encode(
    x='INSPECTION DATE:T',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
inspections = alt.layer(line, selectors, points, rules, text,
                       data=grades_ts, 
                       width=600, height=400,title='Monthly Restaurant Inspections')
inspections
# inspections.save('inspections_trend.html')

alt.LayerChart(...)

<a id='map_viz'></a>

### Create a map to visualize the locations of the restaurants that received letter C inspection grade. We will use the NYC NTA boundary mapping. 

Since there is a limitation of how many records can be rendered on a viz (5,000 rows) on Altair, we decided to visualize the location (lat/lng) of restaurants with poor inspection grade (C).

In [28]:
#geometries
nta_topo = 'https://raw.githubusercontent.com/grantpezeshki/NYC-topojson/master/NTA.topojson'
nta = alt.topo_feature(nta_topo, 'collection')

# C-grade restaurant locations 
latest_inspection = df_rest.sort_values(['CAMIS', 'INSPECTION DATE'], 
                    ascending=[True, False]).groupby('CAMIS').head(1)
c_grade = latest_inspection[latest_inspection['GRADE'] == 'C']

# filtering
input_dropdown = alt.binding_select(options=['Brooklyn','Queens','Staten Island', 'Manhattan', 'Bronx'])
selection = alt.selection_single(fields=['BORO'], bind=input_dropdown, name='Borough:')
color = alt.condition(selection,
                    alt.Color('Origin:N', legend=None),
                    alt.value('lightgray'))


# NYC nta background
background = alt.Chart(nta).mark_geoshape(
    stroke='white',
    strokeWidth=2
).encode(
    color=alt.value('#eee'),
).properties(
    width=600,
    height=500
).properties(
    title='Restaurants with Inspection Grade C'
)

#location points
points = alt.Chart(c_grade).mark_circle(
    size=8,
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    tooltip=['DBA', 'CUISINE DESCRIPTION', 'GRADE',
            'BUILDING','STREET','ZIPCODE', 'BORO'],
    color = color
).add_selection(
    selection
)

interactive_map = background + points
interactive_map
#interactive_map.save('Map of C-grade Restaurants.html')

alt.LayerChart(...)

In [23]:
c_grade

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
16086,40369017,PALM TOO,Manhattan,840,2 AVENUE,10017.0,2126975198,American,2019-09-30,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.751554,-73.970927,106.0,4.0,9000.0,1038763.0,1.013370e+09,MN19
48243,40370507,THE PLAYERS,Manhattan,16,GRAMERCY PARK SOUTH,10003.0,2124756116,American,2019-03-19,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.737776,-73.986483,106.0,2.0,5000.0,1017938.0,1.008750e+09,MN21
57244,40376944,TOMOE SUSHI,Manhattan,172,THOMPSON STREET,10012.0,2127779346,Japanese,2019-09-03,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.727712,-74.000256,102.0,1.0,6500.0,1084919.0,1.005250e+09,MN23
14926,40382802,PARNELL'S PUB,Manhattan,350,EAST 53 STREET,10022.0,2127531761,Irish,2019-10-08,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.756238,-73.965959,106.0,4.0,9800.0,1039803.0,1.013450e+09,MN19
307489,40389504,DALLAS BBQ,Manhattan,1265,3 AVENUE,10021.0,2127729393,Barbecue,2019-10-01,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.770262,-73.960218,108.0,4.0,12600.0,1085193.0,1.014270e+09,MN31
76607,40391594,TAKAHACHI RESTAURANT,Manhattan,85,AVENUE A,10009.0,2125056524,Japanese,2019-07-23,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.725166,-73.984191,103.0,2.0,3200.0,1005764.0,1.004328e+09,MN22
39584,40392927,NATIONAL RESTAURANT & CATERING,Brooklyn,273,BRIGHTON BEACH AVE,11235.0,7186461225,Russian,2019-06-28,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.576889,-73.964816,313.0,48.0,36200.0,3326863.0,3.086720e+09,BK19
81776,40394392,PICCOLO ANGOLO,Manhattan,621,HUDSON STREET,10014.0,2122299177,Italian,2019-07-24,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.738048,-74.005618,102.0,3.0,7900.0,1011419.0,1.006250e+09,MN23
45228,40396892,PARADISE ALLEY BAR,Queens,4109,150 STREET,11355.0,7184606941,American,2019-08-07,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.762389,-73.813815,407.0,20.0,116700.0,4114918.0,4.050580e+09,QN51
12800,40398045,MIKES PIZZA,Manhattan,415,SECOND AVENUE,10010.0,2125325196,Pizza/Italian,2019-07-30,Violations were cited in the following area(s).,...,2019-11-25,Cycle Inspection / Re-inspection,40.738346,-73.980579,106.0,2.0,6400.0,1078779.0,1.009040e+09,MN21


Count the number C restaurants by cuisine type

In [13]:
#count number C restaurants by cuisine type
latest_inspection = df_rest.sort_values(['CAMIS', 'INSPECTION DATE'], 
                    ascending=[True, False]).groupby('CAMIS').head(1)
c_grade = latest_inspection[latest_inspection['GRADE'] == 'C']
c_cuis_cnt = c_grade.groupby('CUISINE DESCRIPTION')['CAMIS']\
            .count().reset_index().sort_values('CAMIS', ascending=False)

#Grade C Grades by cuisine type
bars = alt.Chart(c_cuis_cnt).mark_bar(color ='steelblue').encode(
    x=alt.X('CAMIS:Q', 
    axis=alt.Axis(title ='# of C Grades')),

    y=alt.Y('CUISINE DESCRIPTION:N',
            sort='-x',
            axis=alt.Axis(title ='Cuisine')
    )

)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='CAMIS:Q'
)

cusin_bar = (bars + text).properties(height=500,width=100)
cusin_bar

alt.LayerChart(...)

### Combine the interactive map with bar chart

In [15]:
alt.hconcat(interactive_map ,cusin_bar)

alt.HConcatChart(...)

<a id='top_10'></a>
### Percentage of Inspection Grades for the top 10 cuisine (by count and inspection grade)

In [16]:
#top 10 cuisine by counts
latest_inspection = df_rest.sort_values(['CAMIS', 'INSPECTION DATE'], 
                    ascending=[True, False]).groupby('CAMIS').head(1)
cusines = latest_inspection.groupby(['CUISINE DESCRIPTION'])['CAMIS'].count().reset_index()
topcusines = cusines.sort_values('CAMIS', ascending=False).iloc[:20,:]
topcusines

bars = alt.Chart(topcusines).mark_bar(color ='steelblue').encode(
    x=alt.X('CAMIS:Q', 
    axis=alt.Axis(title ='# of Restaurants')),

    y=alt.Y('CUISINE DESCRIPTION:N',
            sort='-x',
            axis=alt.Axis(title ='Cuisine')
    )

)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  
).encode(
    text='CAMIS:Q'
)

top10cuisine = (bars + text).properties(height=500,width=200)

#Top 10 cusine types by inspection grades 
top10cuis_insp = latest_inspection[latest_inspection['CUISINE DESCRIPTION']\
                  .isin(topcusines['CUISINE DESCRIPTION'].to_list())]\
                    .groupby(['CUISINE DESCRIPTION', 'GRADE'])['CAMIS'].count().reset_index()

top10grade = alt.Chart(top10cuis_insp).mark_bar().encode(
                        x=alt.X('sum(CAMIS)', stack="normalize"),
                        y='CUISINE DESCRIPTION',
                        color='GRADE').properties(
                        height=500,
                        width=200
                    )

alt.hconcat(top10cuisine, top10grade)


alt.HConcatChart(...)